# Introduction

The data was gathered around May 2019, utilizing the Steam Store and SteamSpy APIs, as stated by the dataset's author on the description.

This is an exploratory analysis of this dataset, mainly utilizing Pandas and its functions. I will try to answer the following questions:

- Top 20 games with the most positive ratings
- Top 20 games with the most negative ratings
- Top 20 games with the highest average playtime
- Price analysis
- Distribution of genres
- Publishers with the highest number of games
- Games released by year
- What does ownership look like

In [2]:
import pandas as pd 
 


## **Initial Dataset Examination**

In [3]:
steam_data = pd.read_csv('steam.csv')

In [4]:
print(steam_data)

         appid                        name release_date  english  \
0           10              Counter-Strike   2000-11-01        1   
1           20       Team Fortress Classic   1999-04-01        1   
2           30               Day of Defeat   2003-05-01        1   
3           40          Deathmatch Classic   2001-06-01        1   
4           50   Half-Life: Opposing Force   1999-11-01        1   
...        ...                         ...          ...      ...   
27070  1065230             Room of Pandora   2019-04-24        1   
27071  1065570                   Cyber Gun   2019-04-23        1   
27072  1065650            Super Star Blast   2019-04-24        1   
27073  1066700  New Yankee 7: Deer Hunters   2019-04-17        1   
27074  1069460                   Rune Lord   2019-04-24        1   

                developer             publisher          platforms  \
0                   Valve                 Valve  windows;mac;linux   
1                   Valve                 V

Getting a summary of the statistics in my dataset

In [5]:
summary_stats = steam_data.describe()

In [6]:
print(summary_stats)

              appid       english  required_age  achievements  \
count  2.707500e+04  27075.000000  27075.000000  27075.000000   
mean   5.962035e+05      0.981127      0.354903     45.248864   
std    2.508942e+05      0.136081      2.406044    352.670281   
min    1.000000e+01      0.000000      0.000000      0.000000   
25%    4.012300e+05      1.000000      0.000000      0.000000   
50%    5.990700e+05      1.000000      0.000000      7.000000   
75%    7.987600e+05      1.000000      0.000000     23.000000   
max    1.069460e+06      1.000000     18.000000   9821.000000   

       positive_ratings  negative_ratings  average_playtime  median_playtime  \
count      2.707500e+04      27075.000000      27075.000000      27075.00000   
mean       1.000559e+03        211.027147        149.804949        146.05603   
std        1.898872e+04       4284.938531       1827.038141       2353.88008   
min        0.000000e+00          0.000000          0.000000          0.00000   
25%        6.0

Verifying how many lines and columns my dataset has

In [8]:
num_rows, num_cols = steam_data.shape

Checking if there are any null values in my dataframe. This is good practice, although this dataset was already cleaned by the author.

In [9]:
null_counts = steam_data.isnull().sum()

Columns appid and steamspy_tags will be dropped. The former because it doesn't have any relevant information for this exploratory analysis, and the latter because it has very similar data to the genres column, and that is the chosen column to be used in this analysis.

In [ ]:
steam_data.drop(columns=['appid', 'steamspy_tags'], inplace=True)

Altering the genres column inplace. Since this column has many genres separated by semicolons, we will disregard the ones that come after the first and consider that the main genre.

In [10]:
steam_data['genres'] = steam_data['genres'].apply(lambda x: x.split(';')[0])

There's a lot of zeroes in the column required_age, so a query is run to check whether that is the situation for the entire column or not, in case it's bad data

In [11]:
required_age_counts = steam_data['required_age'].value_counts()

## **Top 20 Games with the Most Positive Ratings**

Discovering the top 20 games with the most positive reviews on Steam. First, the 20 largest values in the positive_ratings column will be filtered.

In [13]:
top_positive = steam_data.nlargest(20, 'positive_ratings')[['name', 'positive_ratings']]

Then, a query is applied to select only those games whose quantity of positive ratings is above the last number retrieved in the filter. In this case, that would be 144595.

In [14]:
threshold = 144595
top_positive_games_above_threshold = steam_data[steam_data['positive_ratings'] > threshold]

Unsurprisingly, the podium is occupied by the wildly popular PC games Counter-Strike: Global Offensive, Dota 2 and Team Fortress 2, all three developed by Steam's own parent company, Valve Software.

## **Top 20 Games with the Most Negative Ratings**

Now, going to the opposite end, discovering the top 20 games with the most negative reviews on Steam. Filtering for the the 20 largest values in the negative_ratings column.

In [15]:
top_negative = steam_data.nlargest(20, 'negative_ratings')[['name', 'negative_ratings']]

Applying a query to select only those games whose quantity of negative ratings is above the last number retrieved in the filter, 25920.

In [16]:
threshold = 25920
top_negative_games_above_threshold = steam_data[steam_data['negative_ratings'] > threshold][['name', 'negative_ratings']]

Popular video games, especially those with live-service and multiplayer components, can often be very divisive or controversial due to the fickle nature of the player bases and the state of their satisfaction or dissatisfaction with the games as they get updated and changed. Thus Counter-Strike: Global Offensive and Dota 2 occupy 2 of the 3 spots of games with the most negative reviews.

## **Top 20 Games with the Highest Average Playtime**

Taking a look at how much time people spend on Steam's most popular games, on average.

First, the average_playtime column will be converted from minutes to hours for better visualization

In [17]:
steam_data['average_playtime_hours'] = steam_data['average_playtime'] / 60
top_avg_playtime = steam_data.nlargest(20, 'average_playtime_hours')[['name', 'average_playtime_hours']]
print("\nTop 20 Games with the Highest Average Playtime:")
print(top_avg_playtime)


Top 20 Games with the Highest Average Playtime:
                                          name  average_playtime_hours
9201               The Abbey of Crime Extensum             3177.083333
1478                 The Banner Saga: Factions             1587.416667
6014      The Secret of Tremendous Corporation             1587.366667
8969                                     PRICE             1058.016667
3930                                 Boundless              920.066667
3969    Shroud of the Avatar: Forsaken Virtues              910.300000
2435                                X-Plane 11              736.150000
23409                                    懒人修仙传              727.200000
2023                           Fantasy Grounds              717.900000
8919                                   Screeps              646.750000
12195                     The Price of Freedom              600.483333
744                   FINAL FANTASY XIV Online              481.616667
3803                        

## **Price Analysis**

A simple price analysis will be shown below, focusing on the split of free vs. paid games, average price by year and general price distribution. 

release_year and release_month columns will be created from the release_date one, as it will be used for this price analysis and a few other metrics down the line

In [18]:
steam_data['release_date'] = pd.to_datetime(steam_data['release_date'], errors='coerce')
steam_data['release_year'] = steam_data['release_date'].dt.year
steam_data['release_month'] = steam_data['release_date'].dt.month

A new boolean column will be added called free, this column displays whether a game is free or paid

In [19]:
steam_data['free'] = steam_data['price'].apply(lambda x: True if x == 0 else False)
print("\nFirst few rows with 'free' column:")
print(steam_data[['name', 'price', 'free']].head())


First few rows with 'free' column:
                        name  price   free
0             Counter-Strike   7.19  False
1      Team Fortress Classic   3.99  False
2              Day of Defeat   3.99  False
3         Deathmatch Classic   3.99  False
4  Half-Life: Opposing Force   3.99  False


## **Publishers and Developers with the Highest Number of Games**

In [20]:
top_publishers = steam_data['publisher'].value_counts().head(10)

In [21]:
top_developers = steam_data['developer'].value_counts().head(10)

Top 10 games by ownership

In [22]:
def clean_owners(owners_str):
    return int(owners_str.replace(',', '').split('-')[0])

steam_data['owners'] = steam_data['owners'].apply(clean_owners)
top_ownership_games = steam_data.nlargest(10, 'owners')[['name', 'owners']]

print("\nTop 10 Games by Ownership:")
print(top_ownership_games)


Top 10 Games by Ownership:
                                   name     owners
22                               Dota 2  100000000
25     Counter-Strike: Global Offensive   50000000
12836     PLAYERUNKNOWN'S BATTLEGROUNDS   50000000
19                      Team Fortress 2   20000000
1634                           Warframe   20000000
3362                           Unturned   20000000
0                        Counter-Strike   10000000
7        Counter-Strike: Condition Zero   10000000
9                           Half-Life 2   10000000
10               Counter-Strike: Source   10000000
